This notebook can be run to generate CSVs necessary for the dashboard graphs. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os

#os.chdir("/content/drive/My Drive/BT4103/Final Submission/Data") Change to the directory which stores the source data file
data = pd.read_pickle('hashed_0_data.h5') 
data

,group,hashed_post_url,hashed_post_id,hashed_user_id,hashed_username,hashed_user_url,text,post_text,time,post_text_pred,post_text_pred_prob,is_live,factcheck,likes,comments,shares,hashed_shared_post_url,hashed_shared_post_id,hashed_shared_user_id,hashed_shared_username,shared_text,shared_time,hashed_reactors,reactions,reaction_count,commenter_meta,hashed_comment_id,hashed_commenter_name,comment_text,comment_time,comment_text_pred,comment_text_pred_prob,reply_commenter_meta,hashed_reply_comment_id,hashed_reply_commenter_name,hashed_reply_commenter_url,reply_comment_text,reply_comment_time,reply_comment_text_pred,reply_comment_text_pred_prob
0,abolishceca,311a45afbc37fcfed1697e59901981578f8574e7c75290...,b0b67a11d931f5511a9c3009542cec3b6cf991fa0f6e4c...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,9fe8f29321bec69fb001c82449645beac7b932172cd0b4...,"#我们的靖康之辱\nWhat now,we responsible for the viru...","#我们的靖康之辱\nWhat now,we responsible for the viru...",2021-05-18 23:36:41,racist,0.947050,False,None,0.0,8.0,4.0,3b9e0d995bc2db815fc166371c2c3bac2ca48bbed001e7...,1eb3b8c33b4990293e489770bb5846edb96387e59044da...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,Michael Fang\n18 May at 15:33 ·\n\n#我们的靖康之辱\nW...,2021-05-18 15:33:00,0,NaN,NaN,NaN,c2e0481cbf2f9737428f179f1f107f8dcb197dc8075c0d...,9d22627881fb31d13e81b0d9b4fa06fd197028809f806e...,Ohhh yesss.. Singapore is such a xyz country t...,NaT,import,0.988628,NaN,0,0,0,NaN,NaT,NaN,NaN
1,abolishceca,311a45afbc37fcfed1697e59901981578f8574e7c75290...,b0b67a11d931f5511a9c3009542cec3b6cf991fa0f6e4c...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,9fe8f29321bec69fb001c82449645beac7b932172cd0b4...,"#我们的靖康之辱\nWhat now,we responsible for the viru...","#我们的靖康之辱\nWhat now,we responsible for the viru...",2021-05-18 23:36:41,racist,0.947050,False,None,0.0,8.0,4.0,3b9e0d995bc2db815fc166371c2c3bac2ca48bbed001e7...,1eb3b8c33b4990293e489770bb5846edb96387e59044da...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,Michael Fang\n18 May at 15:33 ·\n\n#我们的靖康之辱\nW...,2021-05-18 15:33:00,0,NaN,NaN,NaN,8b42806d9f46f87722628aadaa56d01b16d671d3e20512...,62eb887c987eeaa529574103efbad0d6569d0a70deec3f...,Arvin talkcock even their citizens fails for c...,NaT,tyrannical,0.985442,NaN,0,0,0,NaN,NaT,NaN,NaN
2,abolishceca,311a45afbc37fcfed1697e59901981578f8574e7c75290...,b0b67a11d931f5511a9c3009542cec3b6cf991fa0f6e4c...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,9fe8f29321bec69fb001c82449645beac7b932172cd0b4...,"#我们的靖康之辱\nWhat now,we responsible for the viru...","#我们的靖康之辱\nWhat now,we responsible for the viru...",2021-05-18 23:36:41,racist,0.947050,False,None,0.0,8.0,4.0,3b9e0d995bc2db815fc166371c2c3bac2ca48bbed001e7...,1eb3b8c33b4990293e489770bb5846edb96387e59044da...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,Michael Fang\n18 May at 15:33 ·\n\n#我们的靖康之辱\nW...,2021-05-18 15:33:00,0,NaN,NaN,NaN,e0c2c7c0524306f778f325d8fac2c3f4287a0761005ba5...,98f34f7b19b6450e45b90561c8e10439158b15fae4b6d4...,I just want to see the pic/video of them teari...,NaT,racist,0.856327,NaN,0,0,0,NaN,NaT,NaN,NaN
3,abolishceca,311a45afbc37fcfed1697e59901981578f8574e7c75290...,b0b67a11d931f5511a9c3009542cec3b6cf991fa0f6e4c...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,9fe8f29321bec69fb001c82449645beac7b932172cd0b4...,"#我们的靖康之辱\nWhat now,we responsible for the viru...","#我们的靖康之辱\nWhat now,we responsible for the viru...",2021-05-18 23:36:41,racist,0.947050,False,None,0.0,8.0,4.0,3b9e0d995bc2db815fc166371c2c3bac2ca48bbed001e7...,1eb3b8c33b4990293e489770bb5846edb96387e59044da...,b5ebbf99f938a2a3f2ea9c029ef2ff2ad5084b1e0c6f19...,67a48950f4526b35c7c222ab49295646a8ac70be0d7a8f...,

In [ ]:
!pip install pycld3
import cld3

#Determine which texts should be removed before computing mean sentiment score
def notValid(text):
    filtered = False
    if not text:
        return True
    #Filter non-English texts
    if text and cld3.get_language(text).language != 'en':
        filtered = True
    #Also remove texts that have length <= 3
    if len(text.split(' ')) <= 3:
        filtered = True
    return filtered

     |████████████████████████████████| 13.6 MB 16 kB/s 


Generate time_elapsed_filtered.csv, which removes invalid comments that were made prior to post creation date

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from datetime import timedelta

#Remove null post id
post_data = data[data['hashed_post_id'] != 0]

post_data=post_data.dropna(subset=['post_text','time'])

#Get the latest information (likes, comments, shares) for each post
post_latest=post_data.loc[post_data.groupby('hashed_post_id').time.idxmax()].reset_index(drop = True)

#Get the oldest time recorded for each post (actual post creation date)
post = post_data.loc[post_data.groupby('hashed_post_id').time.idxmin()].reset_index(drop = True)

#Add the oldest post_times to the post data
post_new = pd.merge(post_latest.loc[:, post_latest.columns != 'time'], post[['hashed_post_id', 'time']], on = 'hashed_post_id')
post_new = post_new.rename(columns = {'time':"post_time"})
post_new['comment_text'] = ''
post_new['hashed_comment_id'] = 0
post_new = post_new[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id', 'post_text','post_text_pred', 'hashed_username',
                'hashed_comment_id','hashed_commenter_name','comment_text','post_time','comment_time','comment_text_pred','comment_text_pred_prob']]

#Add the oldest post_times to the full dataset
data_new = pd.merge(post_data, post[['hashed_post_id', 'time']], on = 'hashed_post_id')
data_new = data_new.rename(columns = {'time_y':"post_time"})


#Clean up the data by removing null entries and duplicates, and concatenate replies and comments together

comment_data = data_new[data_new['hashed_comment_id'] != 0]
comment_data=comment_data.dropna(subset=['comment_text','comment_time'])
comment_data=comment_data[comment_data['comment_text']!= ""]
comment_sorted=comment_data.loc[comment_data.groupby('hashed_comment_id').comment_time.idxmin()].reset_index(drop = True)
comment_sorted=comment_sorted[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id', 'post_text','post_text_pred', 'hashed_username',
                'hashed_comment_id','hashed_commenter_name','comment_text','post_time','comment_time','comment_text_pred','comment_text_pred_prob']]
            

reply_data = comment_data[comment_data['hashed_reply_comment_id'] != 0]
reply =reply_data.dropna(subset=['reply_comment_text','reply_comment_time'])
reply =reply[reply['reply_comment_text']!= ""]
reply=reply.loc[reply.groupby('reply_comment_text').reply_comment_time.idxmin()].reset_index(drop = True)
reply=reply[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id','post_text','post_text_pred', 'hashed_username',
                'hashed_reply_comment_id','hashed_reply_commenter_name','reply_comment_text', 'post_time',
                'reply_comment_time','reply_comment_text_pred','reply_comment_text_pred_prob']]
reply = reply.rename(columns = {"hashed_reply_comment_id":"hashed_comment_id", "hashed_reply_commenter_name":"hashed_commenter_name", "reply_comment_text":"comment_text",
                "reply_comment_time":"comment_time", "reply_comment_text_pred":"comment_text_pred", "reply_comment_text_pred_prob":"comment_text_pred_prob"})

responses = pd.concat([comment_sorted, reply]).reset_index(drop = True)


#Remove comments which occur before the post time
responses_new = responses[responses['comment_time']>=responses['post_time']]

#Calculate time elapsed before each comment was made
responses_new['time_elapsed'] = responses_new['comment_time'] - responses_new['post_time']

#Concatenate with post data
post_new['time_elapsed'] = timedelta(0)

post_new=post_new.dropna(subset=['post_text'])

#final_df = pd.concat([post_new, responses_new]).reset_index(drop = True)
#Sentiment Analysis

sid = SentimentIntensityAnalyzer()

post_new['sentiment'] = post_new['post_text'].apply(lambda x:sid.polarity_scores(x)['compound'])
responses_new['sentiment'] = responses_new['comment_text'].apply(lambda x:sid.polarity_scores(x)['compound'])

#First, get the 'uncertain' predictions with sentiment == 0
post_score = post_new[post_new['sentiment'] == 0][['post_text', 'sentiment']]
#Among these rows, get those that contain non-English characters, or have length <= 3
post_score = post_score[post_score["post_text"].apply(notValid)]
post_new['include'] = 1
post_new.loc[post_score.index, 'include'] = 0

#Get sentiment score for comments
comment_score = responses_new[responses_new['sentiment'] == 0][['comment_text', 'sentiment', 'comment_text_pred']]
comment_score = comment_score[comment_score["comment_text"].apply(notValid)]
responses_new['include'] = 1
responses_new.loc[comment_score.index, 'include'] = 0

final_df = pd.concat([post_new, responses_new]).reset_index(drop = True)
final_df.to_csv('time_elapsed_filtered.csv', index = False)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Generate time_elapsed.csv, which addresses the problem of early comments, by setting the affected post dates to be equal to the earliest corresponding comment date

In [ ]:

#Remove null post id
post_data = data[data['hashed_post_id'] != 0]

post_data=post_data.dropna(subset=['post_text','time'])

#Get the latest information (likes, comments, shares) for each post
post_latest=post_data.loc[post_data.groupby('hashed_post_id').time.idxmax()].reset_index(drop = True)

#Get the oldest time recorded for each post (actual post creation date)
post = post_data.loc[post_data.groupby('hashed_post_id').time.idxmin()].reset_index(drop = True)

#Add the oldest post_times to the post data
post_new = pd.merge(post_latest.loc[:, post_latest.columns != 'time'], post[['hashed_post_id', 'time']], on = 'hashed_post_id')
post_new = post_new.rename(columns = {'time':"post_time"})
post_new['comment_text'] = ''
post_new['hashed_comment_id'] = 0
post_new = post_new[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id', 'post_text','post_text_pred', 'hashed_username',
                'hashed_comment_id','hashed_commenter_name','comment_text','post_time','comment_time','comment_text_pred','comment_text_pred_prob']]

#Add the oldest post_times to the full dataset
data_new = pd.merge(post_data, post[['hashed_post_id', 'time']], on = 'hashed_post_id')
data_new = data_new.rename(columns = {'time_y':"post_time"})


#Clean up the data by removing null entries and duplicates, and concatenate replies and comments together

comment_data = data_new[data_new['hashed_comment_id'] != 0]
comment_data=comment_data.dropna(subset=['comment_text','comment_time'])
comment_data=comment_data[comment_data['comment_text']!= ""]
comment_sorted=comment_data.loc[comment_data.groupby('hashed_comment_id').comment_time.idxmin()].reset_index(drop = True)
comment_sorted=comment_sorted[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id', 'post_text','post_text_pred','hashed_username',
                'hashed_comment_id','hashed_commenter_name','comment_text','post_time','comment_time','comment_text_pred','comment_text_pred_prob']]
            

reply_data = comment_data[comment_data['hashed_reply_comment_id'] != 0]
reply =reply_data.dropna(subset=['reply_comment_text','reply_comment_time'])
reply =reply[reply['reply_comment_text']!= ""]
reply=reply.loc[reply.groupby('reply_comment_text').reply_comment_time.idxmin()].reset_index(drop = True)
reply=reply[['group','likes','comments','shares','reactions','reaction_count','hashed_post_id','post_text','post_text_pred','hashed_username',
                'hashed_reply_comment_id','hashed_reply_commenter_name','reply_comment_text', 'post_time',
                'reply_comment_time','reply_comment_text_pred','reply_comment_text_pred_prob']]
reply = reply.rename(columns = {"hashed_reply_comment_id":"hashed_comment_id", "hashed_reply_commenter_name":"hashed_commenter_name", "reply_comment_text":"comment_text",
                "reply_comment_time":"comment_time", "reply_comment_text_pred":"comment_text_pred", "reply_comment_text_pred_prob":"comment_text_pred_prob"})

responses = pd.concat([comment_sorted, reply]).reset_index(drop = True)

#Get the earliest comment time for each post
checkData = responses[['hashed_post_id', 'post_time', 'comment_time']].groupby('hashed_post_id').min().reset_index()
responses_new = pd.merge(responses, checkData[['hashed_post_id', 'comment_time']], on = 'hashed_post_id', suffixes = [None, "_new"])

#Replace the invalid post_times which occur before the earliest comments, with the respective earliest comment times.
responses_new['post_time'] = responses_new.apply(lambda x:x['comment_time_new'] if x['comment_time_new'] < x['post_time'] else x['post_time'], axis = 1)

responses_new['time_elapsed'] = responses_new['comment_time'] - responses_new['post_time']

#Concatenate with post data
post_new['time_elapsed'] = timedelta(0)

post_new=post_new.dropna(subset=['post_text'])

sid = SentimentIntensityAnalyzer()

post_new['sentiment'] = post_new['post_text'].apply(lambda x:sid.polarity_scores(x)['compound'])
responses_new['sentiment'] = responses_new['comment_text'].apply(lambda x:sid.polarity_scores(x)['compound'])

#First, get the 'uncertain' predictions with sentiment == 0
post_score = post_new[post_new['sentiment'] == 0][['post_text', 'sentiment']]
#Among these rows, get those that contain non-English characters, or have length <= 3
post_score = post_score[post_score["post_text"].apply(notValid)]
post_new['include'] = 1
post_new.loc[post_score.index, 'include'] = 0

#Get sentiment score for comments
comment_score = responses_new[responses_new['sentiment'] == 0][['comment_text', 'sentiment', 'comment_text_pred']]
comment_score = comment_score[comment_score["comment_text"].apply(notValid)]
responses_new['include'] = 1
responses_new.loc[comment_score.index, 'include'] = 0

final_df = pd.concat([post_new, responses_new]).reset_index(drop = True)
final_df = final_df.loc[:, final_df.columns != 'comment_time_new']
final_df.to_csv('time_elapsed.csv', index = False)

In [ ]:
final_df

,group,likes,comments,shares,reactions,reaction_count,hashed_post_id,post_text,post_text_pred,hashed_username,hashed_comment_id,hashed_commenter_name,comment_text,post_time,comment_time,comment_text_pred,comment_text_pred_prob,time_elapsed,sentiment,include
0,abolishceca,0.0,2.0,0.0,NaN,NaN,004059a7ab08531562d1411c0ac7cc57598d4db49c92a9...,,agreement,52186fc890abc5853d87582eb5e424ee7fc8074250e50e...,0,932eac01db0bd39c71b88341ccfb1688838b9e8f57bdc3...,,2021-05-15 17:30:55.000000,2021-05-17 21:34:57.130097,insult,0.980654,0 days 00:00:00,0.0000,0
1,abolishceca,0.0,15.0,0.0,NaN,NaN,004321fef27f4d591bd2a66ba49b6a1d87706a9a40bd5c...,UMBRAGE.... 🤣🤣🤣\nWHO IS THIS GUY?\nNOW HE IS F...,insult,52186fc890abc5853d87582eb5e424ee7fc8074250e50e...,0,6ecaffa8b464f62d0e8f09fd9384e0c30ee4b4ec6d59f2...,,2021-05-07 11:07:53.000000,NaT,dehuman,0.936901,0 days 00:00:00,0.0000,1
2,abolishceca,0.0,0.0,0.0,NaN,NaN,004b6aceada83fe4503e0f5b360f13398271a92efc884a...,,agreement,b06c4ea15b6cfc84932cad0568e656b65a95dde506f8a6...,0,0,,2021-05-09 08:06:48.000000,NaT,NaN,NaN,0 days 00:00:00,0.0000,0
3,abolishceca,0.0,4.0,3.0,NaN,NaN,00a87d4200d4f48d1222e39590775de88f608a9df8c61f...,,agreement,d2640f263f34198a9c2a498274be750f4727991cf63faa...,0,4dfdc5c4f91f1074f0752a99031d8213aa1f025183033b...,,2021-05-17 06:37:20.000000,2021-05-17 17:58:11.864749,tyrannical,0.944027,0 days 00:00:00,0.0000,0
4,limtean,958.0,230.0,129.0,"{'like': 958, 'love': 14, 'haha': 16, 'wow': 8...",1198.0,0138d2041d0164227f29389f2ac8cffc9b101340f3b04b...,There Is A Lot Of Truth Here!\nAs I said in my...,tyrannical,5e90ebe75bc246ddfe64c8c6a78242806c6d1e218645d6...,0,29fb62d9bff9d6a7fe53d3b2e7288cb24e57ce37228817...,,2019-11-12 07:01:00.000000,2019-05-28 15:14:47.310505,import,0.801723,0 days 00:00:00,0.9462,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40288,limtean,52.0,1.0,17.0,"{'like': 52, 'love': 6, 'care': 1}",59.0,503cc32d899b8254e039dd467e71bd5d8b5b56f9af1470...,My Pasir Ris Punggol GRC constituency broadcas...,others,1e3add59a8b0afb46085393e4f75dfe12b638c709d423c...,fb64160649a75e8bfa2a9053bbe1f29027c25cb4c5d69e...,258cc0eb30b94d0804f4a64b486d70caf471dfc2d72406...,Arvin Kumar Krishnakumar,2020-06-29 19:59:21.240321,2020-06-29 19:59:21.240321,others,0.862616,0 days 00:00:00,0.0000,0
40289,abolishceca,0.0,1.0,0.0,NaN,NaN,ba010222d875fa7dd74ed16b5cba8a49b55abfc4b12656...,Guah guah guah!!!,agreement,927e585f09f3d8ef14b1238fe8a7c575f43bd93e476c0d...,fba9c75854a1333bece63ad250d403343f523365e8de90...,ea54c5038d1be1c2dc5476c4f3364644c6f57bc7a85b70...,That why whole world is banning them ma. Only ...,2021-05-05 13:10:58.000000,2021-05-11 10:16:10.131226,import,0.944398,5 days 21:05:12.131226,0.0000,1
40290,abolishceca,0.0,1.0,0.0,NaN,NaN,9e0b02d975dc6912e8f495bbf32f302a76343ed5fec14b...,,agreement,454c6faffd3ef0bfb80b1fc83e029734f83bd1032cbfaa...,fdadbdfcad75d820459c4c7e7ab5f41164c37e3888700d...,3e3ba97a347d59cd75d21a1fd041c322f20c25d0399f1c...,Short of cecaporean la,2021-05-04 11:08:30.765319,2021-05-04 11:08:30.765319,import,0.988841,0 days 00:00:00,0.0000,0
40291,abolishceca,0.0,1.0,0.0,NaN,NaN,2e48d0b45cb429309f25764e865441270cdb16c0c2f25e...,Complain to new delhi. You think they care?,tyrannical,ab0d7f306ad086b547d1d8d8417056fbd512775453e73f...,fe046c429ad7c1366a2aea294fe99e6ce023513de7fdb3...,ffce171f134e97416fc4bf614937c4a8acaff8fa348448...,"The more u afraid of them, the more they screw u",2021-05-19 00:00:00.000000,2021-05-19 00:00:00.000000,tyrannical,0.691206,0 days 00:00:00,-0.1725,1
